web scraping assignment 3

In [1]:
import pandas as pd
import selenium
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import requests
from selenium.webdriver.common.by import By

from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
import warnings
warnings.filterwarnings('ignore')


Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(3)


In [3]:
#opening homepage of Amazon.in
driver.get("https://www.amazon.in/")

In [4]:
# input that we want search only
input1=input()

guitar


In [5]:
search_bar=driver.find_element(By.ID,'twotabsearchtextbox')#locating search bar by id
search_bar.clear()
search_bar.send_keys(input1)                              #sending user input to search bar
search_button=driver.find_element(By.XPATH, '//div[@Class="nav-search-submit nav-sprite"]/span/input') #click the button to search bar
search_button.click()

Q2.In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [6]:
# scrape all product urls
product_urls = []
start=0
end=3
for page in range(start,end):#for loop for scrapping 3 page
    url=driver.find_elements(By.XPATH, '//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]') #scraping urls
    for i in url:
        product_urls.append(i.get_attribute("href"))     #appending the urls in product_urls list
    nxt_button=driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')#scraping the next button   
    nxt_button.click()
    time.sleep(2)

In [7]:
len(product_urls)

200

In [8]:
#Make empty lists
Brand=[]
product = []
price = []
availability=[]
ReturnExchange=[]
Delivery=[]

In [9]:
for url in product_urls[:10]: ##loop for every guitar in the list
    driver.get(url)
    time.sleep(2)
    
    try:
        brand= driver.find_element(By.XPATH,'//*[@id="productOverview_feature_div"]/div/table/tbody/tr[1]/td[2]/span')
        Brand.append(brand.text)
    except NoSuchElementException:
        Brand.append('-') 
        
    try:
        exchange = driver.find_element(By.XPATH,'//*[@id="RETURNS_POLICY"]/span/div[2]/a')
        ReturnExchange.append(exchange.text)
    except NoSuchElementException:
        ReturnExchange.append('-') 
    try:
        delivery= driver.find_element(By.XPATH,'//span[@class="a-text-bold"]')
        Delivery.append(delivery.text)
    except NoSuchElementException:
        Delivery.append('-') 
        
    
    try:
        product.append(driver.find_element(By.XPATH,'//span[@id="productTitle"]').text)
    except:
        product.append('-')
    try:
        try:
            price.append(driver.find_element(By.XPATH,'//td[@class="a-span12"]').text)
        except:
            price.append(driver.find_element_by_css_selector('span[class="a-price-whole"]').text)             
    except:
        price.append('-')
    try:
        availability.append( driver.find_element(By.XPATH,'//div[@id="availability"]').text)
    except:
        availability.append('-')

In [10]:
print(len(Brand),len(ReturnExchange),len(Delivery),len(product),len(price),len(availability))

10 10 10 10 10 10


In [11]:
df=pd.DataFrame({'Brand':Brand,'ReturnExchange':ReturnExchange,'Price':price,'Delievery_by':Delivery,'urls':url,'Availability':availability})
df

,Brand,ReturnExchange,Price,Delievery_by,urls,Availability
0,Kadence,7 days Replacement,-,"Tuesday, 27 December",https://www.amazon.in/Intern-INT-38C-Acoustic-...,
1,Kadence,7 days Replacement,-,"Thursday, 29 December",https://www.amazon.in/Intern-INT-38C-Acoustic-...,
2,Medellin,7 days Returnable,-,"Friday, 30 December",https://www.amazon.in/Intern-INT-38C-Acoustic-...,In stock.
3,Intern,7 days Replacement,-,"Wednesday, 28 December",https://www.amazon.in/Intern-INT-38C-Acoustic-...,In stock.
4,Juârez,7 days Replacement,-,"Wednesday, 28 December",https://www.amazon.in/Intern-INT-38C-Acoustic-...,In stock.
5,Intern,7 days Replacement,-,"Wednesday, 28 December",https://www.amazon.in/Intern-INT-38C-Acoustic-...,In stock.
6,Intern,7 days Replacement,-,"Wednesday, 28 December",https://www.amazon.in/Intern-INT-38C-Acoustic-...,In stock.
7,Intern,7 days Replacement,-,"Wednesday, 28 December",https://www.amazon.in/Intern-INT-38C-Acoustic-...,In stock.
8,JUAREZ,7 days Replacement,-,"Tuesday, 27 December",https://www.amazon.in/Intern-INT-38C-Acoustic-...,In stock.
9,Intern,7 days Replacement,-,"Wednesday, 28 December",https://www.amazon.in/Intern-INT-38C-Acoustic-...,In stock.


Q3. 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [13]:
url='https://images.google.com/'
urls = []    # empty list for image urls to download
data = []    # empty list for storing data
search_item = ["Fruits", "Cars", "Machine Learning"]
for item in search_item:
    driver.get(url)  # performing a Google search for our selected items
    time.sleep(3)
    search_bar = driver.find_element(By.TAG_NAME,"input")     # finding search bar on Google page   
    search_bar.send_keys(str(item))      # send search item list    
    search_button =driver.find_element(By.XPATH,"//button[@class='Tg7LZd']").click() # now we will click on search button
    
    # scroll the page in order to generate more images on the website
    for _ in range(500):
        driver.execute_script("window.scrollBy(0,100)")        
        imgs = driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:100]:      # getting only 100 images each
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 300:
        break
    print("Obtaining {0} of {1} images".format(i, 300))
    response=requests.get(urls[i])
    file = open(r"C:\Users\91782\Desktop\Fliprobo data/fliprobo image"+str(i)+".jpg", "wb")
    file.write(response.content)

Obtaining 0 of 300 images
Obtaining 1 of 300 images
Obtaining 2 of 300 images
Obtaining 3 of 300 images
Obtaining 4 of 300 images
Obtaining 5 of 300 images
Obtaining 6 of 300 images
Obtaining 7 of 300 images
Obtaining 8 of 300 images
Obtaining 9 of 300 images
Obtaining 10 of 300 images
Obtaining 11 of 300 images
Obtaining 12 of 300 images
Obtaining 13 of 300 images
Obtaining 14 of 300 images
Obtaining 15 of 300 images
Obtaining 16 of 300 images
Obtaining 17 of 300 images
Obtaining 18 of 300 images
Obtaining 19 of 300 images
Obtaining 20 of 300 images
Obtaining 21 of 300 images
Obtaining 22 of 300 images
Obtaining 23 of 300 images
Obtaining 24 of 300 images
Obtaining 25 of 300 images
Obtaining 26 of 300 images
Obtaining 27 of 300 images
Obtaining 28 of 300 images
Obtaining 29 of 300 images
Obtaining 30 of 300 images
Obtaining 31 of 300 images
Obtaining 32 of 300 images
Obtaining 33 of 300 images
Obtaining 34 of 300 images
Obtaining 35 of 300 images
Obtaining 36 of 300 images
Obtaining 3

Obtaining 298 of 300 images
Obtaining 299 of 300 images


Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [20]:
driver=webdriver.Chrome(r'C:\Users\HP\Downloads\chromedriver_win32/chromedriver.exe')
driver.get("https://www.flipkart.com/")
time.sleep(2)
driver.maximize_window()
search = driver.find_element("xpath","/html/body/div[2]/div/div/button").click()
product = input(" Enter the name of Smartphone that has to be searched : ")
search_product=driver.find_element(By.CLASS_NAME,'_3704LK').send_keys(product)

 Enter the name of Smartphone that has to be searched : Oneplus Nord


In [26]:
# Fetching urls of phones coming on 1st page
page1_urls = []
urls = driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for url in urls:
    page1_urls.append(url.get_attribute("href"))

In [27]:
#  creating  empty  list
Brand = []
Phone_name = []
Colour = []
RAM= []
ROM = []
Primary_Camera= []
Secondary_Camera = []
Display_Size= []
Battery_Capacity = []
Price = []
URL = []

In [28]:
# Scraping data from each url of page 1
for url in page1_urls:
    driver.get(url)              
    URL.append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button
    try:
        read_more = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
        
    
    #Scraping brand name of phone data
    try:
        brand_tags = driver.find_element(By.XPATH,'//span[@class="B_NuCI"]')      
        Brand.append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Brand.append('-')
    
    
    #Scraping phone name data
    try:
        name_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')     
        Phone_name.append(name_tags.text)
    except NoSuchElementException:
        Phone_name.append('-')
    
    
    #Scraping phone color data
    try:
        color_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Colour.append(color_tags.text)
    except NoSuchElementException:
        Colour.append('-')
     
    
    #Scraping RAM data
    try:
        ram_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        RAM.append(ram_tags.text)
    except NoSuchElementException:
        RAM.append('-')
    
    
    #Scraping ROM data
    try:
        rom_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        ROM.append(rom_tags.text)
    except NoSuchElementException:
        ROM.append('-')
        
        
    #Scraping Primary camera data
    try:                                                                                    
        pri_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Primary_Camera.append(pri_tags.text)
    except NoSuchElementException:
        Primary_Camera.append('-')
        
        
    #Scraping secondary camera data
    try:                                                                                    
        sec_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Secondary_Camera.append(sec_cam.text)
    except NoSuchElementException:
        Secondary_Camera.append('-')
        
     #Scraping Display size data 
    try:
        disp_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Displa Size" : raise NoSuchElementException
        disp_size = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Display_Size.append(disp_size.text)
    except NoSuchElementException:
        Display_Size.append('-')
        
        #Scraping battery capacity data
    try:
        if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Battery_Capacity.append(bat_capa.text)
    except NoSuchElementException:
        Battery_Capacity.append('-')        
        
    #Scraping Price data
    try:
        price_tags = driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')      
        Price.append(price_tags.text)
    except NoSuchElementException:
        Price.append('-')

In [29]:
smartphone=pd.DataFrame({"brand_name":Brand[:24],"smartphone_name":Phone_name[:24],"product_colour":Colour[:24],"product_RAM":RAM[:24],
                         "product_ROM":ROM[:24],"Primary_Camera":Primary_Camera[:24],"Secondary_Camera":Secondary_Camera[:24],
                         "Display_size":Display_Size[:24],"Battery_Capacity":Battery_Capacity[:24],"Phone_Price":Price[:24],"Product_URL":URL[:24]})

smartphone

,brand_name,smartphone_name,product_colour,product_RAM,product_ROM,Primary_Camera,Secondary_Camera,Display_size,Battery_Capacity,Phone_Price,Product_URL
0,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,Primary Camera,-,-,-,"₹19,949",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,Primary & Secondary Camera,-,-,-,"₹19,945",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,Primary & Secondary Camera,-,-,-,"₹19,890",https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,OnePlus,Nord CE 2 Lite,Black Dusk,6 GB,128 MB,Primary Camera,-,-,-,"₹19,885",https://www.flipkart.com/oneplus-nord-ce-2-lit...
4,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,64MP Rear Camera,16MP Front Camera,-,5000 mAh,"₹19,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
5,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,Primary & Secondary Camera,-,-,-,"₹19,800",https://www.flipkart.com/oneplus-nord-ce-2-lit...
6,OnePlus,Nord CE 2 5G,Bahama Blue,8 GB,128 GB,Primary Camera,-,-,-,"₹24,998",https://www.flipkart.com/oneplus-nord-ce-2-5g-...
7,OnePlus,Nord CE 5G,Charcoal Ink,6 GB,128 GB,Primary Camera,-,-,-,"₹20,899",https://www.flipkart.com/oneplus-nord-ce-5g-ch...
8,OnePlus,Nord 2T 5G,Jade Fog,12 GB,256 GB,Primary & Secondary Camera,-,-,-,"₹33,999",https://www.flipkart.com/oneplus-nord-2t-5g-ja...
9,OnePlus,Nord 2T 5G,Gray Shadow,8 GB,128 GB,Primary Camera,-,-,-,"₹28,999",https://www.flipkart.com/oneplus-nord-2t-5g-gr...


Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [30]:
#Enter City name for find latitude, longitude
City = input("Enter the city name: ")

#connect to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")

# find on google ,google maps 
driver.get("https://www.google.co.in/maps")
time.sleep(2)

#Write city name which is enter by user
search_title = driver.find_element(By.ID,'searchboxinput')
search_title.send_keys(City)

#Click on search button for find city on google map
search_btn = driver.find_element(By.XPATH,'//div[@class="pzfvzf"]')
search_btn.click()
time.sleep(2)

try:
    #get url current url or link
    link = driver.current_url
    print("Current Link  is:",link)
    #now apply regex for filter url or link after @ sign
    filter_link = re.findall(r'@(.*)',link)
    #now split it in a parts so we can Identify it
    x=filter_link[0].split(',')
    # zeroth index of x is a latitude and first index is longitude
    print('The latitude of this',City,'is',x[0])
    print('The longitude of this',City,'is',x[1])
    
except Exception as e:
    print("Error:",str(e))

Enter the city name: delhi
Current Link  is: https://www.google.co.in/maps/place/Delhi/@28.622848,77.0768896,14z/data=!4m5!3m4!1s0x390cfd5b347eb62d:0x37205b715389640!8m2!3d28.7040592!4d77.1024902
Error: name 're' is not defined


6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.[skip]

7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [31]:
#connect to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")
driver.maximize_window()
# find on google ,https://www.digit.in/ 
driver.get("https://www.digit.in/")
time.sleep(2)

#Search for Laptop
search_Laptop = driver.find_element(By.XPATH,'//div[@class="menu"]/ul/li[3]/a')
search_Laptop.click()
time.sleep(0.5)

#Search for best gaming in laptop
search_gaming = driver.find_element(By.XPATH,"//a[text()='Best Gaming Laptops in India']")
search_gaming.click()
time.sleep(2)


#Fetch all require data from webpage
LaptopName =[]
OS=[]
display=[]
processor=[]
memory=[]
weight_dimension=[]
graphics=[]
description=[]

try:
    
    laptop=driver.find_elements(By.XPATH,'//div[@class="left_side"]/a')
    for i in laptop:
        LaptopName.append(i.text)
except NoSuchElementException:
    pass

try:
    
    os=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[5]/td[3]')
    for i in os:
        OS.append(i.text)
except NoSuchElementException:
    pass

try:
    
    displays=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[4]/td[3]')
    for i in displays:
        display.append(i.text)
except NoSuchElementException:
    pass

try:
    
    processors=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[3]/td[3]')
    for i in processors:
        processor.append(i.text)
except NoSuchElementException:
    pass

try:
    
    memorys=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[6]/td[3]')
    for i in memorys:
        memory.append(i.text)
except NoSuchElementException:
    pass

try:
    
    graphicss=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[7]/td[3]')
    for i in graphicss:
        graphics.append(i.text)
except NoSuchElementException:
    pass

try:
    
    weights_dimensions=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[8]/td[3]')
    for i in weights_dimensions:
        weight_dimension.append(i.text)
except NoSuchElementException:
    pass

try:
    
    descriptions=driver.find_elements(By.XPATH,'//div[@class="Section-center"]/div[2]')
    for i in descriptions:
        description.append(i.text)
except NoSuchElementException:
    pass


#Creating DataFrame
df=pd.DataFrame({
    'LaptopName':LaptopName,
    'OS':OS,
    'display':display,
    'processor':processor,
    'memory':memory,
    'weight_dimension':weight_dimension,
    'graphics':graphics,
    'description':description
})

df

,LaptopName,OS,display,processor,memory,weight_dimension,graphics,description
0,MSI Titan GT77-12UHS,Windows 11 Home,"17.3″ (3840 x 2160) screen, 120 Hz refresh rate",12th Gen Intel Core i9-12900HX 16 core processor,64 GB DDR5 RAM & 2 TB SSD,397 x 330 x 23 mm dimension & 3.3 kg weight,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,The very best MSI gaming laptop featuring an I...
1,Alienware X17 R2,Windows 11 Home,"17.3″ (1920 x 1080) screen, 360 Hz refresh rate",12th Gen Intel Core i9-12900H 14 core processo...,32 GB DDR5 RAM & 1 TB SSD,299 x 359 x 21 mm dimension & 2.96 kg weight,16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics...,"Alienware, the premium gaming brand from Dell ..."
2,Acer Predator Triton 500 SE PT516-52s,Windows 11,16″ (2560 x 1600) screen,12th Gen Intel Core i7-12700H 14 core processo...,32 GB DDR5 RAM & 2 TB SSD,262 x 359 x 20 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTX 3070 Ti Graphics ...,The Predator lineup from Acer has shown great ...
3,Omen By HP (16-B1371TX),Windows 11 Home,16.1″ (2560 x 1440) screen,12th Gen Intel Core i7-12700H 14 core processo...,8 GB DDR5 RAM & 1 TB SSD,369 x 248 x 23 mm dimension & 2.32 kg weight,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,Omen by HP is a premium gaming lineup which is...
4,Acer Predator Helios 300 AN515-45,Windows 11 Home,"15.6″ (2560 x 1440) screen, 165 Hz refresh rate",AMD Ryzen 9-5900HX 8 core processor,16 GB DDR4 RAM & 512 GB SSD,363 x 255 x 23.9 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,Acer Predator Helios 300 AN515-45 is the best ...
5,MSI Delta 15 (A5EFK-083IN),Windows 11 Home,"15.6″ (1920 x 1080) screen, 240Hz refresh rate",AMD 5th Gen Ryzen 9-5900HX 8 core processor wi...,16 GB DDR4 RAM & 1 TB SSD,357 x 247 x 19 mm dimension & 1.9 kg weight,10 GB DDR6 AMD Radeon RX 6700M Graphics card,The best MSI gaming laptop with an AMD process...
6,Omen By HP (16-C0141AX),Windows 11 Home,16.1″ (2560 x 1440) screen,AMD Ryzen™ 9 5900HX 8 core processor with 3.3 ...,16 GB DDR4 RAM & 1 TB NVMe,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,Another worthy gaming laptop from the Omen lin...
7,Lenovo Legion 5i Pro (82RF00MGIN),Windows 11 Home,"16″ (2560 x 1600) screen, 165 Hz refresh rate",12th Gen Intel Core i7-12700H 14 core processo...,16 GB DDR5 RAM & 1 TB SSD,360 x 264 x 20 mm dimension & 2.49 kg weight,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,Lenovo has gone a little mainstream with their...
8,Alienware M15 R5 Ryzen Edition Icc-C780001win,Windows 11 Home,"15.6″ (1920 x 1080) screen, 165Hz refresh rate",AMD Ryzen R7-5800H 8 core processor with 4.40 ...,16 GB DDR4 RAM & 512 GB SSD,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,Dell not only has gaming laptops under the Del...
9,Lenovo Slim 7 Gen 6 (82K8002JIN),Windows 11 Home,"15.6″ screen, 165 Hz refresh rate",Lenovo Ryzen 7-5800H processor with 3.2 GHz cl...,16 GB DDR4 RAM & 1 TB SSD,356 x 252 x 16 mm dimension & 1.9 kg weight,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,Lenovo has been pushing for thinner gaming lap...


Q8.Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [32]:
driver=webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver_win32/chromedriver.exe") 
time.sleep(2)

# opening web page
driver.get("https://www.forbes.com/billionaires/")
driver.maximize_window()
time.sleep(2)

In [35]:
# Create Empty list
rank = []
name = []
net_worth = []
age = []
citzn = []
source = []
industry = []

#Fetching Rank
rank_tag = driver.find_elements(By.XPATH,"//div[@class='rank']")
for r in rank_tag:
    try:
        rank.append(r.text)
    except NoSuchElementException:
        rank.append("-")
    
#Fetching Name
name_tag = driver.find_elements(By.XPATH,"//div[@class='personName']")
for n in name_tag:
    try:
        name.append(n.text)
    except NoSuchElementException:
        name.append("-")
        
#Fetching Net Worth
netwrth_tag = driver.find_elements(By.XPATH,"//div[@class='netWorth']")
for nt in netwrth_tag:
    try:
        net_worth.append(nt.text)
    except NoSuchElementException:
        net_worth.append("-")
        
        #Fetching Age
age_tag = driver.find_elements(By.XPATH,"//div[@class='age']")
for a in age_tag:
    try:
        age.append(a.text)
    except NoSuchElementException:
        age.append("-")
    
#Fetching Citizenship
cit_tag = driver.find_elements(By.XPATH,"//div[@class='countryOfCitizenship']")
for c in cit_tag:
    try:
        citzn.append(c.text)
    except NoSuchElementException:
        citzn.append("-")
    
#Fetching Source
src_tag = driver.find_elements(By.XPATH,"//span[@class='source-text']")
for s in src_tag:
    try:
        source.append(s.text)
    except NoSuchElementException:
        source.append("-")
    
#Fetching Industry
ind_tag = driver.find_elements(By.XPATH,"//div[@class='category']")
for ind in ind_tag:
        try:
            industry.append(ind.text)
        except NoSuchElementException:
            industry.append("-")

In [36]:
# Length of all the coloumns
len(rank),len(name),len(net_worth),len(age),len(citzn),len(source),len(industry)
(200, 200, 200, 200, 200, 200, 200)

(200, 200, 200, 200, 200, 200, 200)

In [37]:
#create Dataframe
Billionaires =pd.DataFrame({'Rank':rank,'Name':name,'Net Worth':net_worth,'Age': age,'Citizenship/Country':citzn,'Source':source,'Industry':industry})
Billionaires

,Rank,Name,Net Worth,Age,Citizenship/Country,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


Q.9 Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [64]:
driver=webdriver.Chrome('chromedriver.exe')
driver.get("https://www.youtube.com/watch?v=USccSZnS8MQ")
driver.maximize_window()
time.sleep(2)

AttributeError: 'list' object has no attribute 'sleep'

In [ ]:
i=0
while(i<100):
    driver.execute_script("window.scrollBy(0,500)") # scroll down to get more comments
    i+=1
while(i<402):
    driver.execute_script("window.scrollBy(0,5000)") # scroll down to get more comments
    i+=1
    
comment = []
upvote = []
comment_time = []

comment_tag=(driver.find_elements(By.XPATH,'//yt-formatted-string[@id="content-text"]'))
for i in comment_tag:             
    try:
        comment.append(i.text)
    except NoSuchElementException:
        comment.append("-")

upvote_tag=(driver.find_elements(By.XPATH,"//*[@id='vote-count-middle']"))
for i in upvote_tag:             
    try:
        upvote.append(i.text)
    except NoSuchElementException:
        upvote.append("-")

comment_time_tag=(driver.find_elements(By.XPATH,"//*[@id='header-author']/yt-formatted-string/a"))
for i in comment_time_tag:             
    try:
        comment_time.append(i.text)
    except NoSuchElementException:
        comment_time.append("-")

In [ ]:
video=pd.DataFrame({"Comment":comment,"Upvote":upvote,"Comment_ Time":comment_time})
video

Q.10 Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [ ]:
#connect to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")
driver.maximize_window()

# find on google ,https://www.hostelworld.com/
driver.get("https://www.hostelworld.com/")
time.sleep(2)

#Click on bar
bar=driver.find_element(By.XPATH,'//input[@id="location-text-input-field"]')
bar.click()
time.sleep(2)

#search london
search=driver.find_element(By.XPATH,'//input[@id="search-input-field"]')
search.send_keys("London")
time.sleep(5)

#Click on london name
london=driver.find_element(By.XPATH,'//ul[@id="predicted-search-results"]/li[2]/div')
london.click()
time.sleep(2)

#Click on let's go button
clickk=driver.find_element(By.XPATH,'//div[@class="search-button"]')
clickk.click()
time.sleep(5)


# scrape data for distance,links and price
distances=[]
links=[]
price=[]
for X in range(0,4):
    # scroll page
    x=0
    while x<16:
        driver.execute_script("window.scrollBy(0,500)")
        time.sleep(1)
        x=x+1
    print("scrolled done",X)
    
    try:
        distance=driver.find_elements(By.XPATH,'//div[@class="subtitle body-3"]/a/span[1]')
        for i in distance:
            distances.append(i.text)
    except NoSuchElementException:
        distances.append('-')

    try:
        link=driver.find_elements(By.XPATH,'//div[@class="property"]/div/h2/a')
        for i in link:
            links.append(i.get_attribute('href'))
    except NoSuchElementException:
        links.append('-')
    try:
        for p_form in driver.find_elements(By.XPATH,"(//div[@class='price-col'])"):
            price.append(p_form.text)
    except NoSuchElementException:
        price.append('No')
        
    try:
        m=driver.find_element(By.XPATH,'//div[@class="pagination-item pagination-next"]')
        m.click()
    except NoSuchElementException:
        break

        
#the price of private and dorms in single list this code is by forget it 
privatesform=[]
dormsform=[]
for private in range(0,len(price),2):
    privatesform.append(price[private])
for dorms in range(1,len(price),2):
    dormsform.append(price[dorms])
    
# remove unwanted details from price lists     
pvr=[]
dvr=[]
for i in privatesform:
    if i=='No Privates Available':
        pvr.append(i)
    else:
        pvr.append(i[-7:].strip(' ').strip('\n'))
    
for i in dormsform:
    if i=='No Dorms Available':
        dvr.append(i)
    else:
        dvr.append(i[-7:].strip(' ').strip('\n').strip('m\n'))
        
        
print('Part one completed')


#scrape data for  property_descriptions,names,ratings,total_reviews,overall_reviews
property_descriptions=[]
names=[]
ratings=[]
total_reviews=[]
overall_reviews=[]
for x in links:
    try:
        driver.get(x)
        time.sleep(3)
        
        x=0
        while x<10:
            driver.execute_script("window.scrollBy(0,250)")
            time.sleep(1)
            x=x+1
    
        try:
            property_description=driver.find_element(By.XPATH,'//div[@class="description-container"]/div/div[2]')
            property_descriptions.append(property_description.text)
        except NoSuchElementException:
            property_descriptions.append('-')

        try:
            name=driver.find_element(By.XPATH,'//div[@class="title-2"]')
            names.append(name.text)
        except NoSuchElementException:
            names.append('-')

        try:
            rating=driver.find_element(By.XPATH,'//div[@class="score orange big"]')
            ratings.append(rating.text)
        except NoSuchElementException:
            ratings.append('-')

        try:
            total_review=driver.find_element(By.XPATH,'//div[@class="reviews"]')
            total_reviews.append(total_review.text.strip('Total Reviews'))
        except NoSuchElementException:
            total_reviews.append('-')

        try:
            overall_review=driver.find_element(By.XPATH,'//div[@class="keyword"]')
            overall_reviews.append(overall_review.text)
        except NoSuchElementException:
            overall_reviews.append('-')
        
    except TimeoutException:
        property_descriptions.append('-')
        names.append('-')
        ratings.append('-')
        total_reviews.append('-')
        overall_reviews.append('-')
        prs.append('-')
        drs.append('-')
        
# Creating DataFrame
df=pd.DataFrame({
    'names':names,
    'distances':distances,
    'ratings':ratings,
    'total_reviews':total_reviews,
    'privates':pvr,
    'dorms':dvr,
    'overall_reviews':overall_reviews,
    'property_descriptions':property_descriptions
    
})
df